![New Release: Accelerate YOLOv8](assets/yolov8.png)

# Accelerate Ultralytics YOLOv8 with Speedster

Hi and welcome 👋

In this notebook we will discover how in just a few steps you can speed up the response time of deep learning model inference using the Speedster module from the open-source library nebullvm.

With Speedster's latest API, you can speed up models up to 10 times without any loss of accuracy (option A), or accelerate them up to 20-30 times by setting a self-defined amount of accuracy/precision that you are willing to trade off to get even lower response time (option B). To accelerate your model, Speedster takes advantage of various optimization techniques such as deep learning compilers (in both option A and option B), quantization, half accuracy, and so on (option B).

Let's jump to the code.

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

## Setup

### Install Speedster

In [ ]:
!pip install speedster

In [ ]:
!python -m nebullvm.installers.auto_installer --frameworks torch --compilers all

### Install Ultralytics YOLOv8

In [ ]:
!pip install ultralytics

## Load YOLOv8s

In [ ]:
import torch
from ultralytics import YOLO

yolo = YOLO('yolov8s.pt')

Let's load a test dummy data and see the original output

In [ ]:
test_data = torch.randn(1, 3, 640, 640)
yolo.model(test_data)

The original YOLOv8 model return as output a tuple where the first element is a tensor and the second is a list of tensors. Speedster currently supports only models that return only tensors, so we need to create a wrapper to overcome this issue:

In [6]:
class YOLOWrapper(torch.nn.Module):
    def __init__(self, yolo_model):
        super().__init__()
        self.model = yolo_model.model
    
    def forward(self, x, *args, **kwargs):
        res = self.model(x)
        return res[0], *tuple(res[1])
        
model_wrapper = YOLOWrapper(yolo)

## YOLOv8s Optimization with GPU

We can now optimize the model using speedster:

In [ ]:
from speedster import optimize_model

# Provide some input data for the model    
input_data = [((torch.randn(1, 3, 640, 640), ), torch.tensor([0])) for i in range(100)]

# Run Speedster optimization
optimized_model = optimize_model(
  model_wrapper, input_data=input_data, metric_drop_ths=0.1, store_latencies=True
)

We can finally restore the original output format by wrapping the optimized model in a new class:

In [ ]:
class OptimizedYOLO(torch.nn.Module):
    def __init__(self, optimized_model):
        super().__init__()
        self.model = optimized_model
    
    def forward(self, x, *args, **kwargs):
        res = self.model(x)
        return res[0], list(res[1:])
    
optimized_wrapper = OptimizedYOLO(optimized_model)

In [ ]:
optimized_wrapper(test_data.cuda())

## YOLOv8s Optimization with CPU

In [ ]:
from speedster import optimize_model, save_model, load_model
from ultralytics import YOLO

yolo = YOLO('yolov8s.pt')
model_wrapper = YOLOWrapper(yolo)

# Provide some input data for the model    
input_data = [((torch.randn(1, 3, 640, 640), ), torch.tensor([0])) for i in range(100)]

# Run Speedster optimization
optimized_model = optimize_model(
  model_wrapper, input_data=input_data, metric_drop_ths=0.1, store_latencies=True, device="cpu"
)

optimized_wrapper = OptimizedYOLO(optimized_model)

In [ ]:
optimized_wrapper(test_data)

## Save and reload the optimized model

We can easily save to disk the optimized model with the following line:

In [ ]:
save_model(optimized_model, "model_save_path")

We can then load again the model:

In [ ]:
optimized_model = load_model("model_save_path")
optimized_wrapper = OptimizedYOLO(optimized_model)

What an amazing result, right?!? Stay tuned for more cool content from the Nebuly team :) 

<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Join the community </a> |
    <a href="https://nebuly.gitbook.io/nebuly/welcome/questions-and-contributions" target="_blank" style="text-decoration: none;"> Contribute to the library </a>
</center>

<center> 
    <a href="https://github.com/nebuly-ai/nebuly/tree/main/optimization/speedster#key-concepts" target="_blank" style="text-decoration: none;"> How speedster works </a> •
    <a href="https://github.com/nebuly-ai/nebuly/tree/main/optimization/speedster#documentation" target="_blank" style="text-decoration: none;"> Documentation </a> •
    <a href="https://github.com/nebuly-ai/nebuly/tree/main/optimization/speedster#quick-start" target="_blank" style="text-decoration: none;"> Quick start </a> 
</center>